# Perfect indistinguishability

Caratterizzazione alternativa (ma equivalente) della perfect secrecy.


## Esperimento di indistinguishability

PrivK<sup>eav</sup><sub>M,π</sub>

PrivK $\rightarrow$ crittografia chiave privata \
π $\rightarrow$ nome del cifrario $\rightarrow$ π = (Gen, E, D) \
eav $\rightarrow$ eavesdropper $\rightarrow$ attaccante più debole (può soltanto guardare il canale pubblico)\
M $\rightarrow$ Mallory, attaccante

1. M $\rightarrow$ A : x<sub>0</sub>, x<sub>1</sub> &emsp; <i>attaccante M invia due plaintext ad A</i>
2. A : k $\leftarrow$ Gen() &emsp; <i>A genera una chiave con l'algoritmo Gen() $\in$ π</i> \
    &emsp; b $\leftarrow$ {0, 1} &emsp; <i>A genera un bit <b>b</b> uniformemente random</i>
3. A $\rightarrow$ M : y $\leftarrow$ E<sub>k</sub>(x<sub>b</sub>) &emsp; <i>A invia ad M un ciphertext ottenuto da uno dei 2 plaintext (scelto in base a <b>b</b>)</i>
4. M : b' $\leftarrow$ ... &emsp; <i>M vuole calcolare <b>b'</b> in modo che <b>b'=b</b> (vuole indovinare <b>b</b>)</i>

L'esperimento ha successo (PrivK<sup>eav</sup><sub>M,π</sub> = 1) se <b>b'=b</b>

### Cifrario perfectly indistinguishable

π è <b>perfectly indistuinguishable</b> (e anche perfetto) se:

$\forall$ M : Pr(PrivK<sup>eav</sup><sub>M,π</sub> = 1) = 1/2 &emsp; <i><b>per ogni avversario</b> M , <b>la probabilità che l'esperimento abbia successo è 1/2</b> (anche il migliore avversario è costretto a tirare a indovinare come sua migliore strategia)</i>

Qualunque M vuol dire anche l'avversario con HW illimitato e tempo illimitato (irrealistico)

### Esercizio 

P = C = Z<sup>2</sup><sub>26</sub> &emsp; <i>due caratteri in Z<sub>26</sub></i>\
K = Z<sub>26</sub> &emsp;<i>chiave con un singolo carattere</i>\
Gen() : {k $\leftarrow$ K} &emsp; <i>chiave uniformemente random</i>

E<sub>k</sub> (x x') = (x + k)(x' + k) &emsp; <i>la stessa chiave viene utilizzata per cifrare caratteri diversi $\rightarrow$ sicuramente non è un cifrario perfetto</i>

1. M $\rightarrow$ A : x<sub>0</sub>, x<sub>1</sub> &emsp; x<sub>0</sub> = 00 &emsp; x<sub>1</sub> = 01 &emsp; <i>M invia questi due messaggi ad A</i>
2. A : k $\leftarrow$ Gen() &emsp; b$\leftarrow$ {0, 1} &emsp; <i>A genera il bit random <b>b</b></i>
3. A $\rightarrow$ M : y $\leftarrow$ E<sub>k</sub>(x<sub>b</sub>) &emsp; y = zz' &emsp; <i>A fa l'encryption di un messaggio (scelto in base a <b>b</b>) che è composto da due caratteri z e z'</i>
4. M : b' $\leftarrow$ ... &emsp;<i>M deve indovinare <b>b</b></i>

b' = {0 se z=z' ; 1 altrimenti} &emsp;<i>la sua strategia è: dire che è il messaggio 0 (caratteri uguali nel plaintext) se nel ciphertext ci sono i caratteri uguali, 1 altrimenti </i>

Pr(PrivK<sup>eav</sup><sub>M,π</sub> = 1) = 1 &emsp; <i>con questa strategia, la probabilità di indovinare è 1 $\rightarrow$ indovina sempre (passando quei messaggi)</i>

Pr(PrivK<sup>eav</sup><sub>M,π</sub> = 1) &emsp;=&emsp; Pr(b'=0, b=0) + Pr(b'=1, b=1) &emsp;=&emsp; Pr(b'=0 | b=0) Pr(b=0) + Pr(b'=1 | b=1) Pr(b=1)

= 1/2 Pr(b'=0 | b=0) + 1/2 Pr(b'=1 | b=1) &emsp;=&emsp; 1/2 1 + 1/2 1 = 1 $\neq$ 1/2 $\rightarrow$ il cifrario <b>non è</b> perfectly indistinguishable!

# Experiments

<b>IMPORTANT:</b> The following code is based from this: https://github.com/bitbart/crypto-pub/

### Abstract:

In [34]:
import secrets
from abc import ABC, abstractmethod

def int_of_chr(n):
    return ord(n)-ord('a')

def chr_of_int(n):
    return chr(n + ord('a'))

# Abstract class:

class Cipher(ABC):

    @abstractmethod
    def gen(self,n):
        pass

    @abstractmethod
    def enc(self,x,k):
        pass

    @abstractmethod
    def dec(self,y,k):
        pass

    @abstractmethod
    def string_of_key(self,k):
        pass

    @abstractmethod
    def key_of_string(self,s):
        pass
    
    # Alice's operations in the perfect indistinguishability experiment:
    def alice(self,x0,x1):
        k = self.gen()                         # generate key
        print("k = " + ''.join(str(k)))
        b = secrets.choice([0,1])              # generate random bit
        y = self.enc(x1 if b else x0,k)        # encrypt xb = x0 if b=0, x1 if b=1
        return (b,y)

    
class Mallory(ABC):
    
    @abstractmethod
    def plaintexts(): # the plaintexts the adversary chooses
        pass
    
    @abstractmethod
    def guess(y):     # Mallory's strategy guessing the bit
        pass
    
    
class Experiment():
    
    def __init__(self, pi, m, n):
        self.pi = pi # cipher
        self.m = m   # Mallory
        self.n = n   # total number of experiments
        self.s = 0   # number of experiments where Mallory was the winner
        
    def run(self):
        for i in range(self.n): 
            print("\n" + "Experiment #" + str(i+1))

             # M -> A : x0, x1
            (x0,x1) = self.m.plaintexts()
            print("x0 = " + x0)
            print("x1 = " + x1)

             # A -> M : y = Ek(x[b])
            (b,y) = self.pi.alice(x0,x1) # Alice generates the key, the random bit and performs the encryption
            print("b = " + str(b))   
            print("y = " + y)

             # M : bm   
            bm = self.m.guess(y)
            print("bm = " + str(bm))

            if bm==b:
                print("PrivK = 1 (Mallory wins)")
                self.s = self.s+1
            else:
                print("PrivK = 0 (Mallory loses)")

        print("\n" + "Percentage of success: " + str(self.s*100./self.n))

### Uncipher:

In [2]:
class Uncipher(Cipher):

    def gen(self):
        k = secrets.randbelow(26)
        return k

    def enc(self,x,k):
        return x

    def dec(self,y,k):
        return y

    def string_of_key(self,k):
        return chr_of_int(k)

    def key_of_string(self,s):
        return int_of_chr(s)

### Shift cipher in ECB-mode (with uniform keys and plaintexts of arbitrary length):

In [36]:
# Similar to the one defined in 'shift.ipynb'

class ShiftECB(Cipher):

    def gen(self):
        k = secrets.randbelow(26)
        return k

    def enc(self,x,k):
        # Ek(x1 x2 x3 ... xn,k) = (x1+k)%26 (x2+k)%26 (x3+k)%26 ... (xn+k)%26
        return  ''.join(map(lambda n : chr_of_int((int_of_chr(n) + k)%26), x))

    def dec(self,y,k):
        return  ''.join(map(lambda n : chr_of_int((int_of_chr(n) - k)%26), y))

    def string_of_key(self,k):
        return chr_of_int(k)

    def key_of_string(self,s):
        return int_of_chr(s)

In [37]:
# Adversary for the ShiftECB:

class MalloryShiftECB(Mallory):
    
    def plaintexts(self):
        return ("aa","ab")

    def guess(self,y):
        if y[0]==y[1]:
            bm = 0
        else:
            bm = 1
        return bm

In [38]:
# Privk-eav experiment with the ShiftECB:

pi = ShiftECB()
m = MalloryShiftECB()

experiment = Experiment(pi, m, 100) # 100 experiments

experiment.run()


Experiment #1
x0 = aa
x1 = ab
k = 4
b = 0
y = ee
bm = 0
PrivK = 1 (Mallory wins)

Experiment #2
x0 = aa
x1 = ab
k = 0
b = 0
y = aa
bm = 0
PrivK = 1 (Mallory wins)

Experiment #3
x0 = aa
x1 = ab
k = 11
b = 0
y = ll
bm = 0
PrivK = 1 (Mallory wins)

Experiment #4
x0 = aa
x1 = ab
k = 14
b = 1
y = op
bm = 1
PrivK = 1 (Mallory wins)

Experiment #5
x0 = aa
x1 = ab
k = 8
b = 1
y = ij
bm = 1
PrivK = 1 (Mallory wins)

Experiment #6
x0 = aa
x1 = ab
k = 25
b = 1
y = za
bm = 1
PrivK = 1 (Mallory wins)

Experiment #7
x0 = aa
x1 = ab
k = 21
b = 1
y = vw
bm = 1
PrivK = 1 (Mallory wins)

Experiment #8
x0 = aa
x1 = ab
k = 18
b = 0
y = ss
bm = 0
PrivK = 1 (Mallory wins)

Experiment #9
x0 = aa
x1 = ab
k = 16
b = 0
y = qq
bm = 0
PrivK = 1 (Mallory wins)

Experiment #10
x0 = aa
x1 = ab
k = 6
b = 1
y = gh
bm = 1
PrivK = 1 (Mallory wins)

Experiment #11
x0 = aa
x1 = ab
k = 15
b = 1
y = pq
bm = 1
PrivK = 1 (Mallory wins)

Experiment #12
x0 = aa
x1 = ab
k = 13
b = 0
y = nn
bm = 0
PrivK = 1 (Mallory wins)

Expe